## Loading in and Preprocessing the Dataset

In [2]:
#importing necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np
from scikeras.wrappers import KerasClassifier

In [3]:
#loading in and rescaling the data

train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Create ImageDataGenerators with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32, 
    class_mode='binary'
)

# Print the class indices
print("Class indices:", train_generator.class_indices)


Found 3616 images belonging to 2 classes.
Found 1616 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Class indices: {'NORMAL': 0, 'PNEUMONIA': 1}


In [4]:
# Extracting data from generators to use with cross_val_predict
def extract_data(generator):
    X, y = [], []
    # Iterate over the generator to get all batches
    for _ in range(len(generator)):
        x_batch, y_batch = generator.next()  # Get the next batch of images and labels
        X.extend(x_batch)  # Append images to the list
        y.extend(y_batch)  # Append labels to the list
    return np.array(X), np.array(y)  # Convert lists to NumPy arrays

# Extract data and labels from the training generator
X_train_all, y_train_scaled = extract_data(train_generator)


In [6]:
#Checking the distributions of the two classes in the training data
labels = train_generator.classes

# Get the class indices
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

# Calculate the distribution
unique, counts = np.unique(labels, return_counts=True)
distribution = dict(zip(unique, counts))

# Print the distribution
for label, count in distribution.items():
    print(f"Class {class_labels[label]}: {count} images")


Class NORMAL: 541 images
Class PNEUMONIA: 3075 images


## Testing a Baseline Neural Network

In [9]:
from tensorflow.keras import models, layers, regularizers
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the input shape
input_shape = (150, 150, 3)

# Create the baseline model
baseline_model = models.Sequential()
baseline_model.add(layers.Flatten(input_shape=input_shape))
baseline_model.add(layers.Dense(512, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the baseline model on the training data
baseline_model.fit(train_generator, epochs=50, validation_data=val_generator)

# Evaluate the baseline model on the validation data
val_loss, val_accuracy = baseline_model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for the validation data
val_preds = baseline_model.predict(val_generator)
val_preds = (val_preds > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision = precision_score(val_true, val_preds)
val_recall = recall_score(val_true, val_preds)
val_f1 = f1_score(val_true, val_preds)
val_conf_matrix = confusion_matrix(val_true, val_preds)

print("Baseline Model Validation Metrics:")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")
print(f"Confusion Matrix:\n {val_conf_matrix}")


Epoch 1/50
113/113 [==============================] - 117s 1s/step - loss: 5.0046 - accuracy: 0.8473 - val_loss: 0.1864 - val_accuracy: 0.9313
Epoch 2/50
113/113 [==============================] - 108s 959ms/step - loss: 0.3001 - accuracy: 0.9184 - val_loss: 1.3048 - val_accuracy: 0.6677
Epoch 3/50
113/113 [==============================] - 109s 964ms/step - loss: 0.4959 - accuracy: 0.9051 - val_loss: 0.7324 - val_accuracy: 0.8645
Epoch 4/50
113/113 [==============================] - 118s 1s/step - loss: 0.1418 - accuracy: 0.9596 - val_loss: 0.2219 - val_accuracy: 0.9301
Epoch 5/50
113/113 [==============================] - 102s 903ms/step - loss: 0.1464 - accuracy: 0.9593 - val_loss: 0.3754 - val_accuracy: 0.8861
Epoch 6/50
113/113 [==============================] - 99s 877ms/step - loss: 0.1043 - accuracy: 0.9685 - val_loss: 1.0124 - val_accuracy: 0.7580
Epoch 7/50
113/113 [==============================] - 101s 893ms/step - loss: 0.1894 - accuracy: 0.9483 - val_loss: 0.2626 - val_ac

## Adding in Early Stopping

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the input shape
input_shape = (150, 150, 3)

# Create the baseline model
baseline_model = models.Sequential()
baseline_model.add(layers.Flatten(input_shape=input_shape))
baseline_model.add(layers.Dense(512, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the baseline model with early stopping
history = baseline_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the baseline model on the validation data
val_loss, val_accuracy = baseline_model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for the validation data
val_preds = baseline_model.predict(val_generator)
val_preds = (val_preds > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision = precision_score(val_true, val_preds)
val_recall = recall_score(val_true, val_preds)
val_f1 = f1_score(val_true, val_preds)
val_conf_matrix = confusion_matrix(val_true, val_preds)

print("Baseline Model Validation Metrics:")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")
print(f"Confusion Matrix:\n {val_conf_matrix}")


Epoch 1/50
113/113 [==============================] - 101s 884ms/step - loss: 4.3534 - accuracy: 0.8786 - val_loss: 1.6289 - val_accuracy: 0.7884
Epoch 2/50
113/113 [==============================] - 98s 868ms/step - loss: 2.3261 - accuracy: 0.8822 - val_loss: 2.5849 - val_accuracy: 0.8113
Epoch 3/50
113/113 [==============================] - 99s 876ms/step - loss: 0.4151 - accuracy: 0.9497 - val_loss: 3.7244 - val_accuracy: 0.6832
Epoch 4/50
113/113 [==============================] - 99s 877ms/step - loss: 0.4912 - accuracy: 0.9397 - val_loss: 1.5077 - val_accuracy: 0.8205
Epoch 5/50
113/113 [==============================] - 101s 891ms/step - loss: 0.7017 - accuracy: 0.9278 - val_loss: 0.5143 - val_accuracy: 0.9313
Epoch 6/50
113/113 [==============================] - 98s 867ms/step - loss: 0.2838 - accuracy: 0.9571 - val_loss: 1.3539 - val_accuracy: 0.8317
Epoch 7/50
113/113 [==============================] - 99s 877ms/step - loss: 0.3336 - accuracy: 0.9522 - val_loss: 0.3683 - val_

## Testing a L1 Regularized Neural Network

In [11]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Define the input shape
input_shape = (150, 150, 3)

# Create the L1 regularized model
l1_model = models.Sequential()
l1_model.add(layers.Flatten(input_shape=input_shape))
l1_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
l1_model.add(layers.Dense(1, activation='sigmoid'))
l1_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the L1 regularized model with early stopping
history_l1 = l1_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the L1 regularized model on the validation data
val_generator.reset()
val_loss_l1, val_accuracy_l1 = l1_model.evaluate(val_generator)
print(f"L1 Regularization Model Validation Loss: {val_loss_l1}")
print(f"L1 Regularization Model Validation Accuracy: {val_accuracy_l1}")

# Get predictions for the validation data
val_preds_l1 = l1_model.predict(val_generator)
val_preds_l1 = (val_preds_l1 > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision_l1 = precision_score(val_true, val_preds_l1)
val_recall_l1 = recall_score(val_true, val_preds_l1)
val_f1_l1 = f1_score(val_true, val_preds_l1)
val_conf_matrix_l1 = confusion_matrix(val_true, val_preds_l1)

print("L1 Regularization Model Validation Metrics:")
print(f"Precision: {val_precision_l1}")
print(f"Recall: {val_recall_l1}")
print(f"F1 Score: {val_f1_l1}")
print(f"Confusion Matrix:\n {val_conf_matrix_l1}")


Epoch 1/50
113/113 [==============================] - 123s 1s/step - loss: 166.5393 - accuracy: 0.6905 - val_loss: 69.9648 - val_accuracy: 0.5000
Epoch 2/50
113/113 [==============================] - 118s 1s/step - loss: 52.1068 - accuracy: 0.7022 - val_loss: 43.1520 - val_accuracy: 0.5000
Epoch 3/50
113/113 [==============================] - 118s 1s/step - loss: 43.7484 - accuracy: 0.7223 - val_loss: 45.7658 - val_accuracy: 0.5000
Epoch 4/50
113/113 [==============================] - 117s 1s/step - loss: 41.2955 - accuracy: 0.7378 - val_loss: 40.6003 - val_accuracy: 0.5000
Epoch 5/50
113/113 [==============================] - 119s 1s/step - loss: 38.9745 - accuracy: 0.7824 - val_loss: 39.1116 - val_accuracy: 0.5000
Epoch 6/50
113/113 [==============================] - 119s 1s/step - loss: 38.6116 - accuracy: 0.7876 - val_loss: 41.1009 - val_accuracy: 0.5000
Epoch 7/50
113/113 [==============================] - 120s 1s/step - loss: 39.3788 - accuracy: 0.7851 - val_loss: 38.9613 - val_a

## Testing an L2 Regularized Neural Network

In [12]:
#Creating and testing an l2 regularized model
# Define the input shape
input_shape = (150, 150, 3)

# Create the L2 regularized model
l2_model = models.Sequential()
l2_model.add(layers.Flatten(input_shape=input_shape))
l2_model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
l2_model.add(layers.Dense(1, activation='sigmoid'))
l2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the L2 regularized model with early stopping
history_l2 = l2_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the L2 regularized model on the validation data
val_generator.reset()
val_loss_l2, val_accuracy_l2 = l2_model.evaluate(val_generator)
print(f"L2 Regularization Model Validation Loss: {val_loss_l2}")
print(f"L2 Regularization Model Validation Accuracy: {val_accuracy_l2}")

# Get predictions for the validation data
val_preds_l2 = l2_model.predict(val_generator)
val_preds_l2 = (val_preds_l2 > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

val_precision_l2 = precision_score(val_true, val_preds_l2)
val_recall_l2 = recall_score(val_true, val_preds_l2)
val_f1_l2 = f1_score(val_true, val_preds_l2)
val_conf_matrix_l2 = confusion_matrix(val_true, val_preds_l2)

print("L2 Regularization Model Validation Metrics:")
print(f"Precision: {val_precision_l2}")
print(f"Recall: {val_recall_l2}")
print(f"F1 Score: {val_f1_l2}")
print(f"Confusion Matrix:\n {val_conf_matrix_l2}")


Epoch 1/50
113/113 [==============================] - 111s 972ms/step - loss: 11.0169 - accuracy: 0.8440 - val_loss: 10.6716 - val_accuracy: 0.6281
Epoch 2/50
113/113 [==============================] - 108s 959ms/step - loss: 4.5008 - accuracy: 0.8963 - val_loss: 11.6844 - val_accuracy: 0.6553
Epoch 3/50
113/113 [==============================] - 109s 968ms/step - loss: 4.3729 - accuracy: 0.8968 - val_loss: 6.0187 - val_accuracy: 0.7531
Epoch 4/50
113/113 [==============================] - 110s 970ms/step - loss: 2.2799 - accuracy: 0.9234 - val_loss: 7.5765 - val_accuracy: 0.6677
Epoch 5/50
113/113 [==============================] - 109s 961ms/step - loss: 3.1976 - accuracy: 0.8822 - val_loss: 9.2560 - val_accuracy: 0.6139
Epoch 6/50
113/113 [==============================] - 106s 939ms/step - loss: 1.1238 - accuracy: 0.9367 - val_loss: 1.5377 - val_accuracy: 0.8150
Epoch 7/50
113/113 [==============================] - 106s 940ms/step - loss: 0.6808 - accuracy: 0.9270 - val_loss: 1.051

## Testing a Neural Network with Dropout

In [13]:
#Creating and testing a model with dropout

dropout_model = models.Sequential()
dropout_model.add(layers.Flatten(input_shape=input_shape))
dropout_model.add(layers.Dense(512, activation='relu'))
dropout_model.add(layers.Dropout(0.5))
dropout_model.add(layers.Dense(1, activation='sigmoid'))
dropout_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the dropout model with early stopping
history_dropout = dropout_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the dropout model on the validation data
val_generator.reset()
val_loss_dropout, val_accuracy_dropout = dropout_model.evaluate(val_generator)
print(f"Dropout Model Validation Loss: {val_loss_dropout}")
print(f"Dropout Model Validation Accuracy: {val_accuracy_dropout}")

# Get predictions for the validation data
val_preds_dropout = dropout_model.predict(val_generator)
val_preds_dropout = (val_preds_dropout > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

val_precision_dropout = precision_score(val_true, val_preds_dropout)
val_recall_dropout = recall_score(val_true, val_preds_dropout)
val_f1_dropout = f1_score(val_true, val_preds_dropout)
val_conf_matrix_dropout = confusion_matrix(val_true, val_preds_dropout)

print("Dropout Model Validation Metrics:")
print(f"Precision: {val_precision_dropout}")
print(f"Recall: {val_recall_dropout}")
print(f"F1 Score: {val_f1_dropout}")
print(f"Confusion Matrix:\n {val_conf_matrix_dropout}")


Epoch 1/50
113/113 [==============================] - 103s 902ms/step - loss: 3.0378 - accuracy: 0.8639 - val_loss: 0.6589 - val_accuracy: 0.7426
Epoch 2/50
113/113 [==============================] - 101s 892ms/step - loss: 0.3182 - accuracy: 0.8877 - val_loss: 0.3598 - val_accuracy: 0.8434
Epoch 3/50
113/113 [==============================] - 101s 896ms/step - loss: 0.2592 - accuracy: 0.8874 - val_loss: 0.5263 - val_accuracy: 0.6423
Epoch 4/50
113/113 [==============================] - 100s 883ms/step - loss: 0.2434 - accuracy: 0.8866 - val_loss: 0.4568 - val_accuracy: 0.8212
Epoch 5/50
113/113 [==============================] - 98s 870ms/step - loss: 0.2187 - accuracy: 0.8919 - val_loss: 0.4648 - val_accuracy: 0.8063
Epoch 6/50
113/113 [==============================] - 98s 866ms/step - loss: 0.2569 - accuracy: 0.8515 - val_loss: 0.5170 - val_accuracy: 0.5000
Epoch 7/50
51/51 [==============================] - 23s 446ms/step - loss: 0.3598 - accuracy: 0.8434
Dropout Model Validation 

## Testing a Convolutional Neural Network

In [14]:
#Testing a CNN model
input_shape = (150, 150, 3)

# Create the CNN model
cnn_model = models.Sequential()

# Add convolutional and pooling layers
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
cnn_model.add(layers.Flatten())

# Add dense layers
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the CNN model with early stopping
history_cnn = cnn_model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate the CNN model on the validation data
val_generator.reset()
val_loss_cnn, val_accuracy_cnn = cnn_model.evaluate(val_generator)
print(f"CNN Model Validation Loss: {val_loss_cnn}")
print(f"CNN Model Validation Accuracy: {val_accuracy_cnn}")

# Get predictions for the validation data
val_preds_cnn = cnn_model.predict(val_generator)
val_preds_cnn = (val_preds_cnn > 0.5).astype(int)  # Convert probabilities to class labels

# Get true labels from the validation generator
val_true = val_generator.classes

# Calculate additional metrics
val_precision_cnn = precision_score(val_true, val_preds_cnn)
val_recall_cnn = recall_score(val_true, val_preds_cnn)
val_f1_cnn = f1_score(val_true, val_preds_cnn)
val_conf_matrix_cnn = confusion_matrix(val_true, val_preds_cnn)

print("CNN Model Validation Metrics:")
print(f"Precision: {val_precision_cnn}")
print(f"Recall: {val_recall_cnn}")
print(f"F1 Score: {val_f1_cnn}")
print(f"Confusion Matrix:\n {val_conf_matrix_cnn}")


Epoch 1/50
113/113 [==============================] - 141s 1s/step - loss: 0.2808 - accuracy: 0.9060 - val_loss: 0.1569 - val_accuracy: 0.9418
Epoch 2/50
113/113 [==============================] - 128s 1s/step - loss: 0.1101 - accuracy: 0.9577 - val_loss: 0.2843 - val_accuracy: 0.8948
Epoch 3/50
113/113 [==============================] - 129s 1s/step - loss: 0.0653 - accuracy: 0.9748 - val_loss: 0.3116 - val_accuracy: 0.8874
Epoch 4/50
113/113 [==============================] - 130s 1s/step - loss: 0.0465 - accuracy: 0.9831 - val_loss: 0.3380 - val_accuracy: 0.8998
Epoch 5/50
113/113 [==============================] - 129s 1s/step - loss: 0.0341 - accuracy: 0.9859 - val_loss: 0.1660 - val_accuracy: 0.9505
Epoch 6/50
51/51 [==============================] - 26s 507ms/step - loss: 0.1569 - accuracy: 0.9418
CNN Model Validation Loss: 0.15689897537231445
CNN Model Validation Accuracy: 0.9418317079544067
51/51 [==============================] - 27s 520ms/step
CNN Model Validation Metrics:
P

## Testing a Class-Balanced Convolutional Neural Network

In [23]:
# Define and compile the class balanced CNN model (based on the first model you provided)
cnn_model_class_balanced = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

cnn_model_class_balanced.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Train the class balanced CNN model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_class_balanced = cnn_model_class_balanced.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Evaluate the class balanced CNN model on the validation data
val_loss_class_balanced, val_accuracy_class_balanced = cnn_model_class_balanced.evaluate(val_generator)
print(f"Class Balanced CNN Model Validation Loss: {val_loss_class_balanced}")
print(f"Class Balanced CNN Model Validation Accuracy: {val_accuracy_class_balanced}")

# Get predictions for the validation data
val_preds_class_balanced = cnn_model_class_balanced.predict(val_generator)
val_preds_class_balanced = (val_preds_class_balanced > 0.5).astype(int)

# Get true labels from the validation generator
val_true_class_balanced = val_generator.classes

# Calculate additional metrics
val_precision_class_balanced = precision_score(val_true_class_balanced, val_preds_class_balanced)
val_recall_class_balanced = recall_score(val_true_class_balanced, val_preds_class_balanced)
val_f1_class_balanced = f1_score(val_true_class_balanced, val_preds_class_balanced)
val_conf_matrix_class_balanced = confusion_matrix(val_true_class_balanced, val_preds_class_balanced)

print("Class Balanced CNN Model Validation Metrics:")
print(f"Precision: {val_precision_class_balanced}")
print(f"Recall: {val_recall_class_balanced}")
print(f"F1 Score: {val_f1_class_balanced}")
print(f"Confusion Matrix:\n {val_conf_matrix_class_balanced}")


Epoch 1/50
113/113 [==============================] - 143s 1s/step - loss: 0.2881 - accuracy: 0.8722 - val_loss: 0.1417 - val_accuracy: 0.9511
Epoch 2/50
113/113 [==============================] - 129s 1s/step - loss: 0.1366 - accuracy: 0.9502 - val_loss: 0.1843 - val_accuracy: 0.9350
Epoch 3/50
113/113 [==============================] - 130s 1s/step - loss: 0.0999 - accuracy: 0.9627 - val_loss: 0.1464 - val_accuracy: 0.9431
Epoch 4/50
113/113 [==============================] - 129s 1s/step - loss: 0.0790 - accuracy: 0.9696 - val_loss: 0.1516 - val_accuracy: 0.9517
Epoch 5/50
113/113 [==============================] - 129s 1s/step - loss: 0.0607 - accuracy: 0.9751 - val_loss: 0.1317 - val_accuracy: 0.9585
Epoch 6/50
113/113 [==============================] - 129s 1s/step - loss: 0.0385 - accuracy: 0.9848 - val_loss: 0.1553 - val_accuracy: 0.9567
Epoch 7/50
113/113 [==============================] - 129s 1s/step - loss: 0.0385 - accuracy: 0.9856 - val_loss: 0.1728 - val_accuracy: 0.9530

## Testing A Convolutional Neural Network with Added Layers

In [22]:
#Testing A CNN with Added Layers
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = cnn_model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

val_loss, val_accuracy = cnn_model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

val_preds = cnn_model.predict(val_generator)
val_preds = (val_preds > 0.5).astype(int)  

val_true = val_generator.classes

val_precision = precision_score(val_true, val_preds)
val_recall = recall_score(val_true, val_preds)
val_f1 = f1_score(val_true, val_preds)
val_conf_matrix = confusion_matrix(val_true, val_preds)

print("CNN Model Validation Metrics:")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")
print(f"Confusion Matrix:\n {val_conf_matrix}")

Epoch 1/50
113/113 [==============================] - 120s 1s/step - loss: 0.4406 - accuracy: 0.8520 - val_loss: 0.6481 - val_accuracy: 0.6838
Epoch 2/50
113/113 [==============================] - 114s 1s/step - loss: 0.3337 - accuracy: 0.8686 - val_loss: 0.3866 - val_accuracy: 0.8527
Epoch 3/50
113/113 [==============================] - 114s 1s/step - loss: 0.2047 - accuracy: 0.9162 - val_loss: 0.3125 - val_accuracy: 0.8793
Epoch 4/50
113/113 [==============================] - 116s 1s/step - loss: 0.1392 - accuracy: 0.9416 - val_loss: 0.1998 - val_accuracy: 0.9288
Epoch 5/50
113/113 [==============================] - 121s 1s/step - loss: 0.1012 - accuracy: 0.9613 - val_loss: 0.2241 - val_accuracy: 0.9090
Epoch 6/50
113/113 [==============================] - 133s 1s/step - loss: 0.0808 - accuracy: 0.9712 - val_loss: 0.2403 - val_accuracy: 0.9202
Epoch 7/50
113/113 [==============================] - 139s 1s/step - loss: 0.0747 - accuracy: 0.9723 - val_loss: 0.3077 - val_accuracy: 0.9035

## Best Model Evaluation:

In [24]:
# Evaluating the class balanced CNN model on the test data
test_loss_class_balanced, test_accuracy_class_balanced = cnn_model_class_balanced.evaluate(test_generator)
print(f"Class Balanced CNN Model Test Loss: {test_loss_class_balanced}")
print(f"Class Balanced CNN Model Test Accuracy: {test_accuracy_class_balanced}")

test_preds_class_balanced = cnn_model_class_balanced.predict(test_generator)
test_preds_class_balanced = (test_preds_class_balanced > 0.5).astype(int)  # Convert probabilities to class labels

test_true_class_balanced = test_generator.classes

test_precision_class_balanced = precision_score(test_true_class_balanced, test_preds_class_balanced)
test_recall_class_balanced = recall_score(test_true_class_balanced, test_preds_class_balanced)
test_f1_class_balanced = f1_score(test_true_class_balanced, test_preds_class_balanced)
test_conf_matrix_class_balanced = confusion_matrix(test_true_class_balanced, test_preds_class_balanced)

print("Class Balanced CNN Model Test Metrics:")
print(f"Precision: {test_precision_class_balanced}")
print(f"Recall: {test_recall_class_balanced}")
print(f"F1 Score: {test_f1_class_balanced}")
print(f"Confusion Matrix:\n {test_conf_matrix_class_balanced}")


20/20 [==============================] - 9s 460ms/step - loss: 0.6486 - accuracy: 0.8301
Class Balanced CNN Model Test Loss: 0.6486026644706726
Class Balanced CNN Model Test Accuracy: 0.8301281929016113
20/20 [==============================] - 8s 402ms/step
Class Balanced CNN Model Test Metrics:
Precision: 0.6352459016393442
Recall: 0.7948717948717948
F1 Score: 0.7061503416856493
Confusion Matrix:
 [[ 56 178]
 [ 80 310]]
